# Advent of Code

The docstrings are pretty bad.

In [1]:
import os
import re
import math
from datetime import datetime
from typing import List, Tuple, Optional, Any, Callable, Dict, Set
import textwrap
from collections import defaultdict, Counter, deque
from pprint import pprint

from pydantic.dataclasses import dataclass, Field
import requests as req
import pandas as pd
import numpy as np
import numpy.ma as ma
import altair as alt

## Helper Functions

In [2]:
def print_solutions(a1sol, a2sol):
    print(textwrap.dedent(f"""
        a1_solution: {a1sol}
        a2_solution: {a2sol}
    """))

---
## Advent Day 1

In [3]:
data_01 = np.genfromtxt('a01.csv', delimiter=',')

def advent_01a(data: np.ndarray = data_01) -> int:
    """Gives a solution to https://adventofcode.com/2021/day/1, part 1."""
    return np.sum(np.diff(data) > 0)

def advent_01b(data: np.ndarray = data_01) -> int:
    """Gives a solution to https://adventofcode.com/2021/day/1, part 2."""
    lag_array = np.array([
        [data[idx], data[idx + 1], data[idx + 2]] 
        for idx in range(len(data) - 2)
    ])
    lag_array_sum_across = np.sum(lag_array, axis=1)
    return np.sum(np.diff(lag_array_sum_across) > 0)

print_solutions(advent_01a(), advent_01b())


a1_solution: 1696
a2_solution: 1737



---
## Advent Day 2

In [4]:
@dataclass
class XYCoord:
    """XYCoordinate with addition and scalar multiplication."""
    x: int
    y: int
    
    def __add__(self, coord):
        return XYCoord(self.x + coord.x, self.y + coord.y)

    def __mul__(self, other: int):
        return XYCoord(self.x * other, self.y * other)
    
    def __rmul__(self, other: int):
        return self.__mul__(other)
    
    
direction_map = {
    "forward": XYCoord(1, 0),
    "down": XYCoord(0, 1),
    "up": XYCoord(0, -1)
}

def parse_directions_part_1(data: List[str]) -> XYCoord:
    """Parses directions a la https://adventofcode.com/2021/day/2 ."""
    current_loc = XYCoord(0, 0)
    aim = 0
    
    # Split the direction "forward 5", map "forward" to XYCoord, mult by int("5"),
    # and add to current loc.
    for row in data:
        direction, amount = row.split(" ")
        amount = int(amount)
        current_loc += amount * direction_map[direction]

    return current_loc    

def parse_directions_part_2(data: List[str]) -> XYCoord:
    """Parses directions a la https://adventofcode.com/2021/day/2 ."""
    current_loc = XYCoord(0, 0)
    aim = 0
    
    # Split the direction "forward 5", map "forward" to XYCoord, mult by int("5"),
    # and add to current loc.
    for row in data:
        direction, amount = row.split(" ")
        amount = int(amount)        
                
        if direction == "down":
            aim += amount
        elif direction == "up":
            aim -= amount
        elif direction == "forward":
            current_loc += XYCoord(amount, amount * aim)

    return current_loc
        
    
# MAIN
with open("a02.csv", "r") as f:
    data = [line.strip() for line in f.readlines() if line]

def advent_02a(data: List[str] = data) -> int:
    """Solves Advent 2a."""
    dirs = parse_directions_part_1(data)
    return dirs.x * dirs.y

def advent_02b(data: List[str] = data) -> int:
    """Solves Advent 2b."""
    dirs = parse_directions_part_2(data)
    return dirs.x * dirs.y

print_solutions(advent_02a(), advent_02b())


a1_solution: 2117664
a2_solution: 2073416724



---
## Advent Day 3

In [77]:
with open("a03.csv", "r") as f:
    # Open file, create matrix of rows / cols.
    data_03 = [line.strip() for line in f.readlines()]
    
def binary_string_list_to_2D_array(data: List[str]) -> np.ndarray:
    """ Transforms a list of same-sized binary digits into a matrix with
    each value as a row. """
    return np.array([list(map(int, list(row))) for row in data])

def a03a(data: List[str]) -> int:
    """Solves https://adventofcode.com/2021/day/3."""
    data = binary_string_list_to_2D_array(data)
    binary_string_length = len(data[0])
    
    def mode_is_one(row: np.array) -> str:
        """Returns True if 1 is more frequent or THE SAME FREQUENCY as 0."""
        return sum(row) >= len(row) / 2
    
    def int_from_bin_list(s: List[int]) -> int:
        """Returns the int value of a list of binary values."""
        return int("".join(map(str, s)), base=2)
        
    gamma_rate = int_from_bin_list(
        [1 if mode_is_one(data[:, idx]) else 0 for idx in range(len(data[0]))]
    )
    
    epsilon_rate = int_from_bin_list(
        ["0" if mode_is_one(data[:, idx]) else "1" for idx in range(len(data[0]))]
    )

    return gamma_rate * epsilon_rate


class A03b:
    def __init__(self, data: List[str]):
        """Solves https://adventofcode.com/2021/day/3."""
        self.data = binary_string_list_to_2D_array(data)
        self.oxygen_generator_rating = self.data.copy()
        self.co2_scrubber_rating = self.data.copy()
        
        # Main
        self.oxy = self.find_rating(self.oxygen_generator_rating, self.oxy_mode_return)
        self.co2 = self.find_rating(self.co2_scrubber_rating, self.co2_mode_return)

    @staticmethod
    def binary_string_list_to_2D_array(data: List[str]) -> np.ndarray:
        """ Transforms a list of same-sized binary digits into a matrix with
        each value as a row. """
        return np.array([list(map(int, list(row))) for row in data])
        
    def int_from_bin_list(self, s: List[int]) -> int:
        """Returns the int value of a list of binary values."""
        return int("".join(map(str, s)), base=2)
    
    # NOTE: This part is gross.  I should have done something like calculate 
    # s >= data_size / 2 then made oxy / co2 return based on that value...
    def oxy_mode_return(self, s: int, data_size: int) -> bool:
        return 1 if s >= data_size / 2 else 0
    
    def co2_mode_return(self, s: int, data_size: int) -> bool:
        return 0 if s >= data_size / 2 else 1
    
    def find_rating(self, data: np.ndarray, mode_return: Callable) -> np.ndarray:
        rating = data.copy()
        for idx in range(len(rating[:, 0])):
            if len(rating) == 1:
                break
            s = sum(rating[:, idx])
            desired_bit = mode_return(s, len(rating[:, 0]))
            rating = np.array([row for row in rating if row[idx] == desired_bit])
        return self.int_from_bin_list(rating[0])

a03b = A03b(data_03)
print_solutions(a03a(data_03), a03b.oxy * a03b.co2)


a1_solution: 3912944
a2_solution: 4996233



---
## Advent Day 4

In [83]:
class Board:
    def __init__(self, board: np.ndarray):
        self.board = board.copy()
        self.called = np.zeros_like(self.board)
        self.size: int = self.board.shape[0]  # Square
        
    def mark_board(self, lookup_num: int) -> None:
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row, col] == lookup_num:
                    self.called[row, col] = 1
    
    def check_card(self) -> bool:
        for idx in range(self.size):
            if ((sum(self.called[idx, :]) == 5) or
                (sum(self.called[:, idx]) == 5)):
                return True
        return False
        
    def reset_card(self):
        self.called = np.zeros_like(self.board)

In [4]:
def initialize() -> Tuple[Board, np.ndarray]:
    with open("a04_boards.csv", "r") as boards_f:
        boards_txt = boards_f.read()

    boards_raw = [board.split("\n") for board in boards_txt.split("\n\n")]
    boards_raw_parsed = []
    for board in boards_raw:
        board_parsed = []
        for row in board:
            board_parsed.append([int(i) for i in row.split(" ") if i])
        boards_raw_parsed.append(board_parsed)
    boards_raw_parsed = np.array(boards_raw_parsed)

    with open("a04_calls.csv", "r") as calls_f:
        calls = np.genfromtxt(calls_f, delimiter=",", encoding="utf-8")

    boards = [Board(board) for board in boards_raw_parsed]
    
    return boards, calls

def play_bingo() -> int:
    boards, calls = initialize()
        
    def calculate_winning_score(board: Board, call: int):
        mx = ma.masked_array(board.board, mask=board.called)
        return int(call * mx.sum())
        
    for call in calls:
        for board in boards:
            board.mark_board(call)
            if board.check_card():
                return calculate_winning_score(board, call)
            
def lose_at_bingo() -> int:
    boards, calls = initialize()
    non_winning_boards = boards.copy()
    
    def calculate_winning_score(board: Board, call: int):
        mx = ma.masked_array(board.board, mask=board.called)
        return int(call * mx.sum())
        
    winning_board = None
    for call in calls:
        for board in non_winning_boards:
            board.mark_board(call)

        non_winning_boards = [board for board in boards if not board.check_card()]
        if len(non_winning_boards) == 1:
            winning_board = non_winning_boards[0]
        
        if len(non_winning_boards) == 0:
            # The last board won!
            return calculate_winning_score(winning_board, call)

print_solutions(play_bingo(), lose_at_bingo())

NameError: name 'Board' is not defined

---
## Advent Day 5

In [111]:
class Chart:
    
    def __init__(self, chart_size: List[int]):

        self.chart_size = chart_size
        self.chart = np.zeros(shape=self.chart_size)

    def plot(self, coord):
        """Adds one to the location on the chart."""
        self.chart[coord[1], coord[0]] += 1
        
class LineSegment:
    def __init__(self, x1: int, y1: int, x2: int, y2: int, include_diagonal: bool = False):
        self.include_diagonal = include_diagonal
        self.coord = (x1, y1, x2, y2)
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.integer_coords: np.ndarray = None
        
        self.slope = self._compute_slope()
        self.y_intercept = self._compute_y_intercept()
        
        self._compute_coords_on_segment()
        
    def _compute_slope(self):
        if self.x1 == self.x2:
            return None
        return (self.y1 - self.y2) / (self.x1 - self.x2) 
 
        
    def _compute_y_intercept(self):
        if self._compute_slope() == None:
            return None
        return self.y1 - self.slope * self.x1
    
    def _compute_coords_on_segment(self):
        """Computes coords with integer values on the line segments."""
        min_y = min(self.y1, self.y2)
        max_y = max(self.y1, self.y2)
        min_x = min(self.x1, self.x2)
        max_x = max(self.x1, self.x2)
        
        tmp_integer_coords = []
        
        # Line is vertical...
        if self.y_intercept is None:
            for _y in range(min_y, max_y + 1):
                tmp_integer_coords.append([self.x1, _y])
            
        # Line is horizontal...
        elif self.slope == 0:
            for _x in range(min_x, max_x + 1):
                tmp_integer_coords.append([_x, self.y1])
        
        # line is diagonal...
        else:
            if self.include_diagonal:
                for _x in range(min_x, max_x + 1):
                    val = self.slope * _x + self.y_intercept
                    try:
                        tmp_integer_coords.append([_x, int(val)])
                    except:
                        print(f"{val} isn't an integer!")
        
        self.integer_coords = np.array(tmp_integer_coords)

    def plot_integer_coords_on_chart(self, chart: Chart):
        for coord in self.integer_coords:
            chart.plot(coord)
            

In [113]:
def input_parser(data: str) -> List[int]:
    """ Parses input data. """
    coord_list = []
    for row in data.split("\n"):
        row_split = row.split(" -> ")
        coords = row_split[0].split(",") + row_split[1].split(",")
        coords = list(map(int, coords))
        coord_list.append(coords)
    return coord_list

def get_chart_size(coords: List[int]) -> Tuple[int]:
    """ Helper function to size the chart appropriately. """
    coords = np.array(coords)
    # min_x = coords[:, [0, 2]].min()
    max_x = coords[:, [0, 2]].max() + 1
    # min_y = coords[:, [1, 3]].min()
    max_y = coords[:, [1, 3]].max() + 1
    return (max_x, max_y)

def aoc05(include_diagonal: bool = False):
    """ Solves AoC 5.  For a, include_diagonal=False.  For b, it is true. """
    with open("a05.txt", "r") as f:
        data = f.read()

    coords = input_parser(data)
    chart_size = get_chart_size(coords)
    
    ls = [LineSegment(*coord, include_diagonal=include_diagonal) for coord in coords]
    ch = Chart(chart_size)

    for lineseg in ls:
        lineseg.plot_integer_coords_on_chart(ch)

    return (ch.chart >= 2).sum()

print_solutions(aoc05(), aoc05(True))


a1_solution: 6005
a2_solution: 23864



---
## Advent Day 6

In [85]:
class School:
    """ Initial attempt at AoC Day 6.  Brute Force.  Doesn't work for
    part 2! """
    def __init__(self, timers: List[int]):
        self.timers = np.array(timers)
        
    def pass_day(self):
        self.timers -= 1
        moms_mask = self.timers == -1
        new_moms = moms_mask.sum()
        self.timers = self.timers[~moms_mask]
        self.timers = np.append(self.timers, [8, 6] * new_moms)

def parse_input(data: str) -> List[int]:
    return list(map(int, data.split(",")))

def aoc06_a() -> int:
    with open("a06.csv", "r") as f:
        data = f.read()

    timers = parse_input(data)
    school = School(timers)

    days = 80
    ls = []
    for idx in range(days):
        ls.append(len(school.timers))
        school.pass_day()
    
    return len(school.timers)

def aoc06_b() -> int:
    with open("a06.csv", "r") as f:
        data = f.read()

    timers = parse_input(data)
    binned_data = np.bincount(timers, minlength=9)

    def iterate(data: np.ndarray):
        new_data = np.roll(data, shift=-1)
        new_data[6] += data[0]
        return new_data

    for _ in range(256):
        binned_data = iterate(binned_data)

    return binned_data.sum()
    
print_solutions(aoc06_a(), aoc06_b())


a1_solution: 390011
a2_solution: 1746710169834



---
## Advent Day 7

In [77]:
def parse_input(s: str) -> np.ndarray:
    return np.array(list(map(int, s.split(","))))

with open("a07.csv", "r") as f:
    data = parse_input(f.read())
    
def aoc_7a(data: np.ndarray) -> int:
    """Note the median will always produce a minimum value, by definition."""
    return data[math.floor(np.quantile(data, 0.5))]

def aoc_7b(data: np.ndarray) -> int:
    """Brute force."""
    
    def calculate_dynamic_fuel_for_horiz_pos(pos: int, data: np.ndarray) -> int:
        """Tests horizontal position where fuel costs sum(1..n) for n positions moved."""
    
        def vsum_of_digits(n):
            """Vectorized sum of digits."""
            return n * (n + 1) / 2

        return vsum_of_digits(np.absolute(data - pos)).sum()
    
    values = []
    for pos in range(data.min(), data.max() + 1):
        values.append([pos, calculate_dynamic_fuel_for_horiz_pos(pos, data)])

    arr = np.array(values)
    return int(arr[arr[:, 1].argmin()][1])
    
print_solutions(aoc_7a(data), aoc_7b(data))


a1_solution: 187
a2_solution: 99266250



---
## Advent Day 8

In [3]:
# This was a nightmare to try to do without brute-force and with trying to keep to 
# "General" patterns.  In truth, this can be done using a few if-else statements.
# I wanted to practice a bit of my magic methods and the like, so I made some
# objects which turned out to be of fairly limited use, but were fun to make.

ORIG_DIGIT_ENCODINGS = list(map(set, ["abcefg", "cf", "acdeg",  
"acdfg", "bcdf", "abdfe", 
"abdefg", "acf", "abcdefg", 
"abcdfg"]))

class Digit:
    def __init__(self, name: str, segments: str):
        self.name = name
        self.segments = set(segments)
        self.length = len(self.segments)
        
    def __repr__(self):
        return f"Digit({self.name}, {self.segments}, {self.length})"
        
    def __and__(self, other):
        return self.segments.intersection(other.segments)
    
    def __lt__(self, other):
        self.name < other.name
        
    def __lte__(self, other):
        self.name <= other.name
    
def create_digits() -> List[Digit]:
    return [Digit(0, "abcefg"), Digit(1, "cf"), Digit(2, "acdeg"),  
            Digit(3, "acdfg"), Digit(4, "bcdf"), Digit(5, "abdfe"), 
            Digit(6, "abdefg"), Digit(7, "acf"), Digit(8, "abcdefg"), 
            Digit(9, "abcdfg")]

def parse_code_string(code_string: str) -> Tuple[Set, Set]:
    """Parses input of aoc_8."""
    code_string = code_string.split(" | ")
    code, output = code_string

    codes = set(list(map(lambda x: "".join(sorted(x)), code.split(" "))))
    output = list(map(lambda x: "".join(sorted(x)), output.split(" ")))

    return (codes, output)

def determine_1478(codes: Set) -> Tuple[Dict[str, Digit], Dict[str, Digit]]:
    possible_digits = {"".join(sorted(code)): sorted([digit for digit in digits if digit.length == len(code)]) 
                       for code in codes}
    
    matching_matches = {digit: possibilities[0].name
                       for digit, possibilities in possible_digits.items()
                       if len(possibilities) == 1}
    return matching_matches, possible_digits

class DigitIntersectionSignature:
    def __init__(self, signature: str):
        self.signature = signature
        self.intersection_signature = [
            len(signature.intersection(ORIG_DIGIT_ENCODINGS[i]))
            for i in [1, 4, 7, 8]]
        
def decode_by_intersection_signature(code_output):
    c, o = code_output
    matches = determine_1478(c)[0]
    matches_by_number = {v: set(k) for k, v in matches.items()}

    digit_signatures = [DigitIntersectionSignature(sig) for sig in ORIG_DIGIT_ENCODINGS]
    data = np.array(list(digit_signature.intersection_signature for n, digit_signature in enumerate(digit_signatures)))

    encoded_values = defaultdict(int)
    for k in c:
        signature = [len(set(k).intersection(matches_by_number[i])) for i in [1, 4, 7, 8]]
        encoded_values[k] = [n for n, row in enumerate(data) if (np.array(row) == signature).all()][0]

    output_vals = [encoded_values[val] for val in o]
    return int("".join(map(str, output_vals)))

digits = create_digits()

# sample = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
# edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
# fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
# fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
# aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
# fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
# dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
# bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
# egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
# gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

with open("a08.csv", "r") as sample_f:
    sample = sample_f.read()
    
sample_split = sample.split("\n")
code_output_list = [parse_code_string(row) for row in sample_split]

def aoc_8_a() -> int:
    """Solves aoc_8a."""
    values_1478 = defaultdict(int)
    for codes, output in code_output_list:
        matches = determine_1478(codes)[0]
        for item in output:
            if item in matches:
                values_1478[matches[item]] += 1


    return sum(values_1478.values())

def aoc_8_b() -> int:
    return sum(decode_by_intersection_signature(code_output) for code_output in code_output_list)

print_solutions(aoc_8_a(), aoc_8_b())


a1_solution: 488
a2_solution: 1040429



---
## Advent Day 9

In [31]:
data_raw = """
2199943210
3987894921
9856789892
8767896789
9899965678
""".strip().split("\n")

with open("a09.csv", "r") as f:
    data_raw = f.read().strip().split("\n")

data_raw = np.array(list(map(lambda x: [int(y) for y in x], data_raw)))

class SmokeMap:
    def __init__(self, chart: np.ndarray):
        self.chart = chart
        self.n_rows, self.n_cols = self.chart.shape
        
        #!! This needs to be cleared every time.
        # TODO: How do I deal with something like this?
        self.basin_chart = (self.chart.copy() != 9).astype(int)
        
        
    def find_neighbors(self, idx: int, jdx: int) -> List[int]:
        """Finds up-down-left-right neighbors of (idx, jdx)."""
        neighbor_indices = [
            [idx + 1, jdx],
            [idx - 1, jdx],
            [idx, jdx + 1],
            [idx, jdx - 1]
        ]
        
        valid_neighbor_indices = [
            coord for coord in neighbor_indices
            if coord[0] >= 0 and coord[0] < self.n_rows
            and coord[1] >= 0 and coord[1] < self.n_cols
        ]
        
        neighbors = [self.chart[nbidx[0], nbidx[1]] for nbidx in valid_neighbor_indices]
        return neighbors, valid_neighbor_indices

    def test_if_local_min(self, idx: int, jdx: int) -> bool:
        """Compares neighbors to value of (idx, jdx), sees if
        (idx, jdx) is strictly less then all of them."""
        val = self.chart[idx, jdx]
        neighbors = self.find_neighbors(idx, jdx)[0]
        for neighbor in neighbors:
            if neighbor <= val:
                return False
        return True

    def calculate_total_risk_level(self):
        """Gets all local minima, adds one to its value, sums the results."""
        risk_level = 0
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.test_if_local_min(row, col):
                    risk_level += self.chart[row, col] + 1
                    
        return risk_level
    
    
    def find_basin_size(self, idx: int, jdx: int) -> List[Tuple[int, int]]:
        """Finds basin around (idx, jdx) as defined 
        in https://adventofcode.com/2021/day/9#part2."""
        if self.basin_chart[idx, jdx] == 0:
            return 0
        
        basin_size = 1
        self.basin_chart[idx, jdx] = 0
        neighbors = [nbr for nbr in self.find_neighbors(idx, jdx)[1]
                     if self.basin_chart[nbr[0], nbr[1]]]

        for neighbor in neighbors:    
            basin_size += self.find_basin_size(neighbor[0], neighbor[1])

        return basin_size
    
    def calculate_basin_sizes(self):
        """Gets basin sizes."""
        basin_sizes = []
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if size := self.find_basin_size(row, col):
                     basin_sizes.append(size)
                    
        return basin_sizes   

sm = SmokeMap(data_raw)
print_solutions(
    sm.calculate_total_risk_level(), 
    math.prod(sorted(sm.calculate_basin_sizes(), reverse=True)[:3])
)


a1_solution: 478
a2_solution: 1327014



---
## Adent Day 10

In [58]:
CHUNK_DELIMS = [["{", "[", "<", "("], ["}", "]", ">", ")"]]
DELIMS_OPEN = dict(zip(CHUNK_DELIMS[1], CHUNK_DELIMS[0]))
DELIMS_CLOSE = dict(zip(CHUNK_DELIMS[0], CHUNK_DELIMS[1]))

class DelimString:
    
    def __init__(self, line: str):
        self.line = line
        self.stack = []
        
        self.is_incomplete = None
        self.is_corrupt = None
        self.first_illegal_character = None
        
        self._check_stack()
        
    def _clear_stack(self):
        self.stack = []
        
    def _check_stack(self):
        """Checks stack for corruption or incompleteness."""
        self._clear_stack()
        
        for symbol in self.line:
            # If symbol is an end delim, either we have it joining its opening
            # or it is misaligned.
            if symbol in ")}]>":
                if self.stack[-1] != DELIMS_OPEN[symbol]:
                    # print(f"Expected {delims_close[self.stack[-1]]} got {symbol}.")
                    self.is_corrupt = True
                    self.first_illegal_character = symbol
                    break
                else:
                    # Pop the corresponding opening delim.
                    self.stack.pop()
            else:
                # Otherwise, it's an open delim.
                self.stack.append(symbol)
        
        if self.is_corrupt is None:        
            self.is_incomplete = len(self.stack) != 0                 

In [84]:
with open("a10.csv", "r") as f:
    data = [line.strip() for line in f.readlines()]
    
points = {
    ")": 3,
    "]": 57,
    "}": 1197,
    ">": 25137
}

def aoc10_a():
    illegal_characters = []
    for line in data:
        ds = DelimString(line)
        if not ds.is_incomplete:
            if ds.is_corrupt:
                illegal_characters.append(points[ds.first_illegal_character])

    return sum(illegal_characters)

### Part 2.

COMPLETION_POINTS = {
    ")": 1,
    "]": 2,
    "}": 3,
    ">": 4
}

def find_completion(data: List[str]) -> List[str]:
    """Finds delims to complete string."""
    incomplete = []
    for line in data:
        ds = DelimString(line)
        if ds.is_incomplete:
            incomplete.append(ds)

    completions = []
    for ds in incomplete:
        completions.append([DELIMS_CLOSE[symbol] for symbol in ds.stack[::-1]])

    return completions
        
def calculate_completion_score(completion: List[str]) -> int:
    """Calculates completion score a la AoC10."""
    score = 0
    for symbol in completion:
        score *= 5
        score += COMPLETION_POINTS[symbol]

    return score     

def aoc10_b(data: List[str] = data) -> int:
    completions = find_completion(data)
    scores = []
    for completion in completions:
        scores.append(calculate_completion_score(completion))
    return sorted(scores)[int(len(scores) / 2)]

print_solutions(aoc10_a(), aoc10_b())


a1_solution: 436497
a2_solution: 2377613374



---
## Advent Day 11

In [105]:
class OctoMap:
    def __init__(self, data: np.ndarray):
        self.data = data.copy()
        self.n_rows, self.n_cols = self.data.shape
        self.total_flashes = 0
        
    def step(self):
        """ Adds one to each element, checks for flashing until the octopus
            flashing is static, then zeros-out the flashed octos.
        """
        # Reset flash data.
        self.flashed_once = np.zeros_like(self.data)
        self.num_flashes_in_loop = 9999

        self.data += 1
        
        # We continue looping over the array until we check and re-check
        # each octo for flashing.  If none flashed this loop, the loop is
        # over since no other octos could increase in value after that.
        while self.num_flashes_in_loop > 0:
            self.num_flashes_in_loop = 0  # Reset.
            
            for idx in range(self.n_cols):
                for jdx in range(self.n_cols):
                    if self.data[idx, jdx] > 9 and not self.flashed_once[idx, jdx]:
                        self.flashed_once[idx, jdx] += 1
                        self.num_flashes_in_loop += 1
                        self.increment_neighbors(idx, jdx)
                        
            self.total_flashes += self.num_flashes_in_loop
        self.data[self.data > 9] = 0
                    
    
    def increment_neighbors(self, idx: int, jdx: int):
        """Increments neighboring values in a grid by 1, including diagonals."""
        neighbors = [
            [idx + i, jdx + j] for i in [-1, 0, 1] for j in [-1, 0, 1]
            if ((idx + i) >= 0 and (idx + i) < self.n_cols and
                (jdx + j) >= 0 and (jdx + j) < self.n_rows) and
                (not (i == 0 and j == 0))
        ]
        
        for neighbor in neighbors:
            self.data[neighbor[0], neighbor[1]] += 1
        

In [106]:
def aoc11_a() -> int:
    om = OctoMap(data)
    for _ in range(100):
        om.step()
        
    return om.total_flashes

def aoc11_b() -> int:
    om = OctoMap(data)
    idx = 0
    while True:
        idx += 1
        om.step()
        
        # Put a mercy-switch on the idx.
        if om.data.sum() == 0 or idx > 1000:
            break
            
    return idx

# ====================

data_raw = """8271653836
7567626775
2315713316
6542655315
2453637333
1247264328
2325146614
2115843171
6182376282
2384738675"""

def parse_data(data_raw: str) -> np.ndarray:
    return np.array([[int(i) for i in j] for j in data_raw.split("\n")])

data = parse_data(data_raw)

print_solutions(aoc11_a(), aoc11_b())


a1_solution: 1562
a2_solution: 268



---
## Advent Day 12

In [22]:
class CaveSystem:
    
    def __init__(self, data_str: str):
        """
        Series of nodes, including start and end, for the cave system.
        """
        self.nodes = []
        self.neighbors = defaultdict(list)
        self.paths = [["end"]]  # initialize path list.
        self.completed_paths = []
        self.no_more_paths = False

        self._parse_data(data_str)
        
    def _parse_data(self, data_str: str) -> None:
        lines = data_str.split("\n")
        _edges = [line.split("-") for line in lines]
        for edge in _edges:
            self.nodes += [edge[0], edge[1]]  # Dupes filtered below.
            self.neighbors[edge[0]].append(edge[1])
            self.neighbors[edge[1]].append(edge[0])
        self.nodes = list(set(self.nodes))
    
    def step_path(self, can_revisit_one_small_cave = False):
        def _is_small_cave(c: str) -> bool:
            return c.lower() == c
        
        # Start at "end" and work back to "start".
        new_paths = []
        for path in self.paths:
            if path[-1] == "start":
                self.completed_paths.append(path)
                continue
                
            for neighbor in self.neighbors[path[-1]]:
                
                # Do not revisit "end".
                if neighbor == "end" and "end" in path:
                    continue
                    
                # Add a small cave, depending on parameter for multiple visits.
                elif _is_small_cave(neighbor):
                    visited_small_cave_twice = any([path.count(p) >= 2 for p in path if p == p.lower()])
                    visited_this_small_cave = path.count(neighbor) >= 1
                    
                    if visited_this_small_cave and (visited_small_cave_twice or not can_revisit_one_small_cave):
                        continue
                
                new_paths.append(path.copy() + [neighbor])
        
        if not new_paths:
            self.no_more_paths = True
        else:
            self.paths = new_paths
        
        
        
data = """end-MY
MY-xc
ho-NF
start-ho
NF-xc
NF-yf
end-yf
xc-TP
MY-qo
yf-TP
dc-NF
dc-xc
start-dc
yf-MY
MY-ho
EM-uh
xc-yf
ho-dc
uh-NF
yf-ho
end-uh
start-NF"""

def aoc12(data: str, can_revisit_one_small_cave: bool = False) -> int:
    cs = CaveSystem(data)
    while True:
        cs.step_path(can_revisit_one_small_cave)
        if cs.no_more_paths:
            break
    return len(cs.completed_paths)

print_solutions(aoc12(data), aoc12(data, True))


a1_solution: 5076
a2_solution: 145643



---
## Advent Day 13